In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import math
%matplotlib inline

#import warnings
#warnings.filterwarnings("ignore")

/var/folders/n5/f6xjtlr526zbr86__yns243m0000gn/T/ipykernel_56850/2140814706.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# Data Preprocessing

### 1. Book-Crossing Dataset

In [2]:
# Reading the CSV files and skipping the bad lines
books = pd.read_csv('/Users/sanjeevani1109/Desktop/Book_Recommendation_System/Book crossing /BX-Books.csv', sep=';',on_bad_lines = 'skip', encoding='latin-1')
ratings = pd.read_csv('/Users/sanjeevani1109/Desktop/Book_Recommendation_System/Book crossing /BX-Book-Ratings.csv', sep=';',on_bad_lines = 'skip', encoding='latin-1')
users = pd.read_csv('/Users/sanjeevani1109/Desktop/Book_Recommendation_System/Book crossing /BX-Users.csv',sep=';',on_bad_lines = 'skip', encoding='latin-1')

/var/folders/n5/f6xjtlr526zbr86__yns243m0000gn/T/ipykernel_56850/3698374196.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('/Users/sanjeevani1109/Desktop/Book_Recommendation_System/Book crossing /BX-Books.csv', sep=';',on_bad_lines = 'skip', encoding='latin-1')


In [28]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


### Dimensions of the dataset

In [31]:
print("Books Data:   ", books.shape)
print("Ratings Data: ", ratings.shape)
print("Users Data:   ", users.shape)

Books Data:    (271360, 8)
Ratings Data:  (1149780, 3)
Users Data:    (278858, 3)


### 1.1 Books Table

In [32]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271360 non-null  object
 1   Book-Title           271360 non-null  object
 2   Book-Author          271358 non-null  object
 3   Year-Of-Publication  271360 non-null  object
 4   Publisher            271358 non-null  object
 5   Image-URL-S          271360 non-null  object
 6   Image-URL-M          271360 non-null  object
 7   Image-URL-L          271357 non-null  object
dtypes: object(8)
memory usage: 16.6+ MB


In [33]:
books.describe()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
count,271360,271360,271358,271360,271358,271360,271360,271357
unique,271360,242135,102022,202,16807,271044,271044,271041
top,0195153448,Selected Poems,Agatha Christie,2002,Harlequin,http://images.amazon.com/images/P/185326119X.0...,http://images.amazon.com/images/P/185326119X.0...,http://images.amazon.com/images/P/225307649X.0...
freq,1,27,632,13903,7535,2,2,2


In [5]:
books_cols = list(books.columns)
print("Columns: ", books_cols)

Columns:  ['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher', 'Image-URL-S', 'Image-URL-M', 'Image-URL-L']


In [3]:
#Renaming the columns
books.rename(columns={'Book-Title': 'Title', 'Book-Author': 'Author', 'Year-Of-Publication': 'Year'}, inplace=True)
#Printing the Books Table with renamed columns
print(books.head(2))

         ISBN                Title                Author  Year  \
0  0195153448  Classical Mythology    Mark P. O. Morford  2002   
1  0002005018         Clara Callan  Richard Bruce Wright  2001   

                 Publisher                                        Image-URL-S  \
0  Oxford University Press  http://images.amazon.com/images/P/0195153448.0...   
1    HarperFlamingo Canada  http://images.amazon.com/images/P/0002005018.0...   

                                         Image-URL-M  \
0  http://images.amazon.com/images/P/0195153448.0...   
1  http://images.amazon.com/images/P/0002005018.0...   

                                         Image-URL-L  
0  http://images.amazon.com/images/P/0195153448.0...  
1  http://images.amazon.com/images/P/0002005018.0...  


In [4]:
#Dropping the URL columns for the Books table
books.drop(['Image-URL-S', 'Image-URL-M', 'Image-URL-L'], axis=1, inplace=True)
books.head()

,ISBN,Title,Author,Year,Publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company


In [5]:
# Changing all characters to uppercase in ISBN
books['ISBN'] = books['ISBN'].str.upper()

In [6]:
## Checking for null/missing values in the 'books' table
books.isnull().sum() 

ISBN         0
Title        0
Author       2
Year         0
Publisher    2
dtype: int64

##### This means there are 2 missing values in the 'Author' and 'Publisher' columns each.

In [7]:
#Checking which fields have missing values for the 'Author' column
books.loc[books['Author'].isnull(),:]

,ISBN,Title,Author,Year,Publisher
118033,0751352497,A+ Quiz Masters:01 Earth,NaN,1999,Dorling Kindersley
187689,9627982032,The Credit Suisse Guide to Managing Your Perso...,NaN,1995,Edinburgh Financial Publishing


In [6]:
#Filling the missing values
books.at[118033 ,'Author'] = 'Other'
books.at[187689, 'Author'] = 'Other'

In [9]:
#Checking if the missing values were filled
books.loc[books['Author'].isnull()]

,ISBN,Title,Author,Year,Publisher


In [7]:
#Retrieiving the fields where the 'Publisher' column contains null values
books.loc[books['Publisher'].isnull()]

,ISBN,Title,Author,Year,Publisher
128890,193169656X,Tyrant Moon,Elaine Corvidae,2002,NaN
129037,1931696993,Finders Keepers,Linnea Sinclair,2001,NaN


In [8]:
#Filling the missing values
books.at[128890 ,'Publisher'] = 'Other'
books.at[129037, 'Publisher'] = 'Other'

In [12]:
#Checking if the missing values were filled
books.loc[books['Publisher'].isnull()]

,ISBN,Title,Author,Year,Publisher


In [13]:
## Rehecking for null/missing values in the 'books' table
books.isnull().sum() 

ISBN         0
Title        0
Author       0
Year         0
Publisher    0
dtype: int64

##### All missing values have been filled

In [14]:
#Checking for duplicates
print(books.duplicated().sum())

194


##### There are no duplicate values

In [8]:
#Checking the unique set of possible values in the 'Year' column
books['Year'].unique()

array([2002, 2001, 1991, 1999, 2000, 1993, 1996, 1988, 2004, 1998, 1994,
       2003, 1997, 1983, 1979, 1995, 1982, 1985, 1992, 1986, 1978, 1980,
       1952, 1987, 1990, 1981, 1989, 1984, 0, 1968, 1961, 1958, 1974,
       1976, 1971, 1977, 1975, 1965, 1941, 1970, 1962, 1973, 1972, 1960,
       1966, 1920, 1956, 1959, 1953, 1951, 1942, 1963, 1964, 1969, 1954,
       1950, 1967, 2005, 1957, 1940, 1937, 1955, 1946, 1936, 1930, 2011,
       1925, 1948, 1943, 1947, 1945, 1923, 2020, 1939, 1926, 1938, 2030,
       1911, 1904, 1949, 1932, 1928, 1929, 1927, 1931, 1914, 2050, 1934,
       1910, 1933, 1902, 1924, 1921, 1900, 2038, 2026, 1944, 1917, 1901,
       2010, 1908, 1906, 1935, 1806, 2021, '2000', '1995', '1999', '2004',
       '2003', '1990', '1994', '1986', '1989', '2002', '1981', '1993',
       '1983', '1982', '1976', '1991', '1977', '1998', '1992', '1996',
       '0', '1997', '2001', '1974', '1968', '1987', '1984', '1988',
       '1963', '1956', '1970', '1985', '1978', '1973', '1980'

##### There are some noise values in the 'Year' column as seen above - 'DK Publishing Inc' and 'Gallimard'.

In [9]:
books.loc[books['Year'] == 'DK Publishing Inc',:]

,ISBN,Title,Author,Year,Publisher
209538,078946697X,"DK Readers: Creating the X-Men, How It All Beg...",2000,DK Publishing Inc,http://images.amazon.com/images/P/078946697X.0...
221678,0789466953,"DK Readers: Creating the X-Men, How Comic Book...",2000,DK Publishing Inc,http://images.amazon.com/images/P/0789466953.0...


In [10]:
books.at[209538, 'Publisher'] = 'DK Publishing Inc'
books.at[209538, 'Year'] = 2000
books.at[209538, 'Title'] = 'DK Readers: Creating the X-Men, How It All Began (Level 4: Proficient Readers)'
books.at[209538, 'Author'] = 'Michael Teitelbaum'

In [11]:
books.at[221678, 'Publisher'] = 'DK Publishing Inc'
books.at[221678, 'Year'] = 2000
books.at[221678, 'Title'] = 'DK Readers: Creating the X-Men, How Comic Books Come to Life (Level 4: Proficient Readers)'
books.at[221678, 'Author'] = 'James Buckley'

In [12]:
books.loc[books['Year'] == 'Gallimard',:]

,ISBN,Title,Author,Year,Publisher
220731,2070426769,"Peuple du ciel, suivi de 'Les Bergers\"";Jean-M...",2003,Gallimard,http://images.amazon.com/images/P/2070426769.0...


In [13]:
books.at[220731, 'Publisher'] = 'Gallimard'
books.at[220731, 'Year'] = 2003
books.at[220731, 'Title'] = 'Peuple du ciel - Suivi de Les bergers '
books.at[220731, 'Author'] = 'Jean-Marie Gustave Le ClÃ?Â©zio'

In [14]:
#converting year to integer values
books['Year'] = pd.to_numeric(books['Year'], errors='coerce')

#sorting the books according to ascending order
ordered_books = sorted(books['Year'].unique())

print(ordered_books)

[0, 1376, 1378, 1806, 1897, 1900, 1901, 1902, 1904, 1906, 1908, 1909, 1910, 1911, 1914, 1917, 1919, 1920, 1921, 1922, 1923, 1924, 1925, 1926, 1927, 1928, 1929, 1930, 1931, 1932, 1933, 1934, 1935, 1936, 1937, 1938, 1939, 1940, 1941, 1942, 1943, 1944, 1945, 1946, 1947, 1948, 1949, 1950, 1951, 1952, 1953, 1954, 1955, 1956, 1957, 1958, 1959, 1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2008, 2010, 2011, 2012, 2020, 2021, 2024, 2026, 2030, 2037, 2038, 2050]


In [15]:
books.loc[books['Year'] == 0,:]

,ISBN,Title,Author,Year,Publisher
176,3150000335,Kabale Und Liebe,Schiller,0,"Philipp Reclam, Jun Verlag GmbH"
188,342311360X,Die Liebe in Den Zelten,Gabriel Garcia Marquez,0,Deutscher Taschenbuch Verlag (DTV)
288,0571197639,Poisonwood Bible Edition Uk,Barbara Kingsolver,0,Faber Faber Inc
351,3596214629,"Herr Der Fliegen (Fiction, Poetry and Drama)",Golding,0,Fischer Taschenbuch Verlag GmbH
542,8845229041,Biblioteca Universale Rizzoli: Sulla Sponda De...,P Coelho,0,Fabbri - RCS Libri
...,...,...,...,...,...
270794,014029953X,Foe (Essential.penguin S.),J.M. Coetzee,0,Penguin Books Ltd
270913,0340571187,Postmens House,Maggie Hemingway,0,Trafalgar Square
271094,8427201079,El Misterio De Sittaford,Agatha Christie,0,Editorial Molino
271182,0887781721,Tom Penny,Tony German,0,P. Martin Associates


In [16]:
books.loc[books['Year'] <= 1900,:]

,ISBN,Title,Author,Year,Publisher
176,3150000335,Kabale Und Liebe,Schiller,0,"Philipp Reclam, Jun Verlag GmbH"
188,342311360X,Die Liebe in Den Zelten,Gabriel Garcia Marquez,0,Deutscher Taschenbuch Verlag (DTV)
288,0571197639,Poisonwood Bible Edition Uk,Barbara Kingsolver,0,Faber Faber Inc
351,3596214629,"Herr Der Fliegen (Fiction, Poetry and Drama)",Golding,0,Fischer Taschenbuch Verlag GmbH
542,8845229041,Biblioteca Universale Rizzoli: Sulla Sponda De...,P Coelho,0,Fabbri - RCS Libri
...,...,...,...,...,...
270794,014029953X,Foe (Essential.penguin S.),J.M. Coetzee,0,Penguin Books Ltd
270913,0340571187,Postmens House,Maggie Hemingway,0,Trafalgar Square
271094,8427201079,El Misterio De Sittaford,Agatha Christie,0,Editorial Molino
271182,0887781721,Tom Penny,Tony German,0,P. Martin Associates


##### Since the dataset was published in 2004, all years after 2004 are going to be considered invalid. 

In [17]:
num = books.loc[books['Year'] > 2004,:]
print(len(num))

72


In [18]:
books.loc[books['Year'] > 2004,:]

,ISBN,Title,Author,Year,Publisher
3541,0140124373,Red Dwarf,Grant Naylor,2005,Penguin Books Ltd
12782,0380816792,A Rose in Winter,Kathleen E. Woodiwiss,2011,Harper Mass Market Paperbacks
26166,006272004X,The Astrologer's Handbook,Frances Sakoian,2005,HarperResource
30010,068160204X,The Royals,Kitty Kelley,2020,Bausch &amp; Lombard
37487,0671746103,MY TEACHER FRIED MY BRAINS (RACK SIZE) (MY TEA...,Coville,2030,Aladdin
...,...,...,...,...,...
255409,068471809X,To Have and Have Not,Ernest Hemingway,2037,Simon &amp; Schuster
258207,0743484894,The Winter's Tale (Folger Shakespeare Library),William Shakespeare,2005,Washington Square Press
260974,0671740989,FOOTBALL SUPER TEAMS : FOOTBALL SUPER TEAMS,Bill Gutman,2030,Simon &amp; Schuster Children's Publishing
269371,1582344574,"Ninth Life of Louis Drax, The -POSTPONED",Liz Jensen,2005,Bloomsbury USA


In [19]:
# df_book['Year-Of-Publication'].fillna(round(df_book['Year-Of-Publication'].mean()),inplace=True)
# books['Year-Of-Publication'].fillna(round(books['Year-Of-Publication'].median()), inplace=True)

year_list = []
for i in range(len(num)):
    year_list.append(num['ISBN'].iloc[i])
print(year_list)



    
# # Replace values greater than 2006 or equal to 0 with NaN
# books.loc[(books['Year'] > 2006) | (books['Year'] == 0), 'Year'] = np.nan

# # Fill missing values with the mean of the column
# books['Year'].fillna(round(books['Year'].mean()), inplace=True


['0140124373', '0380816792', '006272004X', '068160204X', '0671746103', '1881273156', '0141002999', '0140622454', '0671791990', '0786890754', '0749924659', '0870449842', '0140301690', '0140144463', '068107468X', '0140201092', '0881501719', '0743474783', '0439448948', '0394701658', '0763626392', '3442436893', '157322295X', '0778320820', '0752848291', '0743457226', '0671878654', '0140860363', '0060567341', '0060541490', '006055701X', '0881500461', '074324477X', '0743474198', '1552782603', '155278410X', '0590085417', '006272049X', '088150078X', '0870446924', '0307124533', '1903436346', '0743492064', '0743492056', '0743492072', '0743448987', '1593100175', '0141002980', '0374103747', '0060738189', '1552782662', '074349346X', '0671266500', '0618441433', '0394172116', '0743456874', '0753820099', '068471941X', '0684718022', '0380000059', '0385732376', '0316156108', '0802714277', '0385338082', '0375829849', '0062731327', '0743492080', '068471809X', '0743484894', '0671740989', '1582344574', '0743

In [20]:
# books.loc[(books['Year'] > 2006), 'Year'] = np.nan
# books.loc[(books['Year'] == 0), 'Year'] = np.nan

In [21]:
# from collections import Counter
# count = Counter(books['Year'])
# [k for k, v in count.items() if v == max(count.values())]

count = books['Year'].value_counts()
maxyear = count[count == count.max()].index.tolist()
print(maxyear)

[2002]


In [22]:
#Replacing all values for Publishing years greater than 2004 or equal to 0 with the maximum year
books.loc[books['Year'] > 2004, 'Year'] = 2002
books.loc[books['Year'] == 0, 'Year'] = 2002

In [23]:
books.isnull().sum()

ISBN         0
Title        0
Author       0
Year         0
Publisher    0
dtype: int64

### 1.2 Users Table

In [29]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278858 entries, 0 to 278857
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   User-ID   278858 non-null  int64  
 1   Location  278858 non-null  object 
 2   Age       168096 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 6.4+ MB


In [30]:
users.describe()

,User-ID,Age
count,278858.00000,168096.000000
mean,139429.50000,34.751434
std,80499.51502,14.428097
min,1.00000,0.000000
25%,69715.25000,24.000000
50%,139429.50000,32.000000
75%,209143.75000,44.000000
max,278858.00000,244.000000


In [31]:
users_cols = list(users.columns)
print("Columns: ", users_cols)

Columns:  ['User-ID', 'Location', 'Age']


In [32]:
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [24]:
#Renaming the columns
users.rename(columns={'User-ID': 'UserID'}, inplace=True)
#Printing the Books Table with renamed columns
print(users.head(2))

   UserID                   Location   Age
0       1         nyc, new york, usa   NaN
1       2  stockton, california, usa  18.0


In [25]:
users.isnull().sum()

UserID           0
Location         0
Age         110762
dtype: int64

In [26]:
#Checking for duplicates
print(users.duplicated().sum())

0


In [38]:
users['Age'].unique()

array([35, 18, 17, 61, 26, 14, 25, 19, 46, 55, 32, 24, 20, 34, 23, 51, 31,
       21, 44, 30, 57, 43, 37, 41, 54, 42, 50, 39, 53, 47, 36, 28, 13, 58,
       49, 38, 45, 62, 63, 27, 33, 29, 66, 40, 15, 60, 79, 22, 16, 65, 59,
       48, 72, 56, 67, 80, 52, 69, 71, 73, 78, 64, 12, 74, 75, 76, 68, 11,
       77, 70, 10])

#### Ages that have values nan, less than 8, and more than 90 are invalid 

In [34]:
age_range = users[users['Age'] <= 90] 
age_range = age_range[age_range['Age'] >= 8]
mean = age_range['Age'].mean().round()   
print(mean)
# users.loc[users['Age'] > 80, 'Age'] = mean    #outliers with age grater than 80 are substituted with mean 
# users.loc[users['Age'] < 10, 'Age'] = mean    #outliers with age less than 10 years are substitued with mean
# users['Age'] = users['Age'].fillna(mean)      #filling null values with mean
# users['Age'] = users['Age'].astype(int)       #changing Datatype to int


35.0


In [35]:
users.loc[(users['Age'] > 90) | (users['Age'] < 8),'Age'] = mean
users['Age']=users['Age'].fillna(mean).astype(int)

In [39]:
users['Age'].unique()

array([35, 18, 17, 61, 26, 14, 25, 19, 46, 55, 32, 24, 20, 34, 23, 51, 31,
       21, 44, 30, 57, 43, 37, 41, 54, 42, 50, 39, 53, 47, 36, 28, 13, 58,
       49, 38, 45, 62, 63, 27, 33, 29, 66, 40, 15, 60, 79, 22, 16, 65, 59,
       48, 72, 56, 67, 80, 52, 69, 71, 73, 78, 64, 12, 74, 75, 76, 68, 11,
       77, 70, 10])

In [44]:
users['Location'].unique()

array(['nyc, new york, usa', 'stockton, california, usa',
       'moscow, yukon territory, russia', ...,
       'sergnano, lombardia, italy', 'stranraer, n/a, united kingdom',
       'tacoma, washington, united kingdom'], dtype=object)

#### Retrieval of City, State, and Country values from the Location column of the Users table

In [48]:

loclist = users.Location.str.split(', ') #splitting the location values with ',' as the seperator
city = []           #empty list for storing city values
state = []          #empty list for storing state values
country = []        #empty list for storing country values
count_no_state = 0    
count_no_country = 0
other_list = ['', ' ', 'n/a', ',']

for entry in loclist:
    # Handling city
    if not entry or entry[0] in other_list:
        city.append('other')
    else:
        city.append(entry[0].lower())

    #Handling state
    if len(entry) < 2 or entry[1] in other_list:
        state_val = 'other'
    else:
        state_val = entry[1].lower()
    state.append(state_val)
    if state_val == 'other':
        count_no_state += 1

    # Handling country
    if len(entry) < 3 or entry[2] in other_list:
        country_val = 'other'
    else:
        country_val = entry[2].lower()
    country.append(country_val)
    if country_val == 'other':
        count_no_country += 1

In [49]:
#Dropping the original 'Location' column from the Users table
users = users.drop('Location', axis=1)

In [50]:
#creating new data frames for storing the extracted city, state, and country values
df_city = pd.DataFrame([ent.split('/')[0] for ent in city], columns=['City'])
df_state = pd.DataFrame(state, columns=['State'])
df_country = pd.DataFrame(country, columns=['Country'])

In [51]:
#Concatenating the Users data frame with city, state, and country data frames
users = pd.concat([users, df_city, df_state, df_country], axis=1)

In [54]:
print("Number of countries without values:", count_no_country)
print("Number of states without values:", count_no_state)

Number of countries without values: 4666
Number of states without values: 16046


In [56]:
users.duplicated().sum()

0

In [57]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278858 entries, 0 to 278857
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   UserID   278858 non-null  int64 
 1   Age      278858 non-null  int64 
 2   City     278858 non-null  object
 3   State    278858 non-null  object
 4   Country  278858 non-null  object
dtypes: int64(2), object(3)
memory usage: 10.6+ MB


In [58]:
users.describe()

,UserID,Age
count,278858.00000,278858.00000
mean,139429.50000,34.82622
std,80499.51502,10.42094
min,1.00000,10.00000
25%,69715.25000,29.00000
50%,139429.50000,35.00000
75%,209143.75000,35.00000
max,278858.00000,80.00000


In [59]:
users.head()

,UserID,Age,City,State,Country
0,1,35,nyc,new york,usa
1,2,18,stockton,california,usa
2,3,35,moscow,yukon territory,russia
3,4,17,porto,v.n.gaia,portugal
4,5,35,farnborough,hants,united kingdom


In [60]:
#can convert city,state,country to string if required ~~~~~~~

### 1.3 Ratings Table

In [61]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   User-ID      1149780 non-null  int64 
 1   ISBN         1149780 non-null  object
 2   Book-Rating  1149780 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 26.3+ MB


In [62]:
ratings.describe()

,User-ID,Book-Rating
count,1.149780e+06,1.149780e+06
mean,1.403864e+05,2.866950e+00
std,8.056228e+04,3.854184e+00
min,2.000000e+00,0.000000e+00
25%,7.034500e+04,0.000000e+00
50%,1.410100e+05,0.000000e+00
75%,2.110280e+05,7.000000e+00
max,2.788540e+05,1.000000e+01


In [63]:
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [68]:
#Renaming the columns
ratings.rename(columns={'User-ID': 'UserID', 'Book-Rating': 'BookRating'}, inplace=True)
#Printing the Books Table with renamed columns
print(ratings.head(2))

   UserID        ISBN  BookRating
0  276725  034545104X           0
1  276726  0155061224           5


In [70]:
ratings.isnull().sum() 

UserID        0
ISBN          0
BookRating    0
dtype: int64

In [71]:
ratings.duplicated().sum() 

0

In [73]:
print(ratings['UserID'].dtype)
print(ratings['BookRating'].dtype)

int64
int64


In [75]:
# Changing all characters to uppercase in ISBN
ratings['ISBN'] = ratings['ISBN'].str.upper()

In [76]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   UserID      1149780 non-null  int64 
 1   ISBN        1149780 non-null  object
 2   BookRating  1149780 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 26.3+ MB


In [77]:
ratings.head()

,UserID,ISBN,BookRating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


### 1.4 Merging Books, Users, and Ratings tables

In [80]:
bx = pd.merge(books, ratings, on='ISBN')
bx = pd.merge(bx, users, on='UserID')

In [81]:
bx.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1044959 entries, 0 to 1044958
Data columns (total 11 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   ISBN        1044959 non-null  object
 1   Title       1044959 non-null  object
 2   Author      1044959 non-null  object
 3   Year        1044959 non-null  int64 
 4   Publisher   1044959 non-null  object
 5   UserID      1044959 non-null  int64 
 6   BookRating  1044959 non-null  int64 
 7   Age         1044959 non-null  int64 
 8   City        1044959 non-null  object
 9   State       1044959 non-null  object
 10  Country     1044959 non-null  object
dtypes: int64(4), object(7)
memory usage: 87.7+ MB


In [82]:
bx.describe()

,Year,UserID,BookRating,Age
count,1.044959e+06,1.044959e+06,1.044959e+06,1.044959e+06
mean,1.995405e+03,1.405863e+05,2.842596e+00,3.652916e+01
std,7.328172e+00,8.052734e+04,3.856090e+00,1.013319e+01
min,1.376000e+03,2.000000e+00,0.000000e+00,1.000000e+01
25%,1.992000e+03,7.041500e+04,0.000000e+00,3.100000e+01
50%,1.997000e+03,1.412010e+05,0.000000e+00,3.500000e+01
75%,2.001000e+03,2.114260e+05,7.000000e+00,4.100000e+01
max,2.004000e+03,2.788540e+05,1.000000e+01,8.000000e+01


In [83]:
bx.head()

,ISBN,Title,Author,Year,Publisher,UserID,BookRating,Age,City,State,Country
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,2,0,18,stockton,california,usa
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,8,5,35,timmins,ontario,canada
2,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,11400,0,49,ottawa,ontario,canada
3,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,11676,8,35,other,other,other
4,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,41385,0,35,sudbury,ontario,canada


In [85]:
ratings['BookRating'].unique()

array([ 0,  5,  3,  6,  8,  7, 10,  9,  4,  1,  2])

#### As seen above, the explicit ratings provided by the users range from 1-10. However, the implicit ratings are represented by 0. We are going to the split the merged dataset into  two datasets: one for explicit ratings and another for implicit ratings.

In [87]:
# Explicit Ratings Dataset
exp = bx[bx['BookRating'] != 0]
exp = exp.reset_index(drop = True)

In [88]:
exp.shape

(389342, 11)

In [91]:
exp.head()

,ISBN,Title,Author,Year,Publisher,UserID,BookRating,Age,City,State,Country
0,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,8,5,35,timmins,ontario,canada
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,11676,8,35,other,other,other
2,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,67544,8,30,toronto,ontario,canada
3,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,116866,9,35,ottawa,other,other
4,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,123629,9,35,kingston,ontario,canada


In [89]:
# Implicit Ratings Dataset
imp = bx[bx['BookRating'] == 0]
imp = imp.reset_index(drop = True)

In [90]:
imp.shape

(655617, 11)

In [92]:
imp.head()

,ISBN,Title,Author,Year,Publisher,UserID,BookRating,Age,City,State,Country
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,2,0,18,stockton,california,usa
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,11400,0,49,ottawa,ontario,canada
2,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,41385,0,35,sudbury,ontario,canada
3,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,85526,0,36,victoria,british columbia,canada
4,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,96054,0,29,ottawa,ontario,canada


### 2. Goodbooks-10k Dataset

In [103]:
goodbooks = pd.read_csv('/Users/sanjeevani1109/Desktop/Book_Recommendation_System/goodreads-10k/books.csv', sep=',',on_bad_lines = 'skip', encoding='latin-1')
goodratings = pd.read_csv('/Users/sanjeevani1109/Desktop/Book_Recommendation_System/goodreads-10k/ratings.csv', sep=';',on_bad_lines = 'skip', encoding='latin-1')
goodbooks_tags = pd.read_csv('/Users/sanjeevani1109/Desktop/Book_Recommendation_System/goodreads-10k/book_tags.csv',sep=';',on_bad_lines = 'skip', encoding='latin-1')
goodtags = pd.read_csv('/Users/sanjeevani1109/Desktop/Book_Recommendation_System/goodreads-10k/tags.csv',sep=';',on_bad_lines = 'skip', encoding='latin-1')
good_toread = pd.read_csv('/Users/sanjeevani1109/Desktop/Book_Recommendation_System/goodreads-10k/to_read.csv',sep=';',on_bad_lines = 'skip', encoding='latin-1')

### Dimensions of the dataset

In [110]:
print("Books Data:    ", goodbooks.shape)
print("Ratings Data:  ", goodratings.shape)
print("Book Tags Data:", goodbooks_tags.shape)
print("Tags Data:     ", goodtags.shape)
print("To Read Data:  ", good_toread.shape)

Books Data:     (10000, 21)
Ratings Data:   (981756, 1)
Book Tags Data: (999912, 1)
Tags Data:      (34252, 1)
To Read Data:   (912705, 1)


### 2.1 Books Table

In [109]:
goodbooks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         10000 non-null  int64  
 1   book_id                    10000 non-null  int64  
 2   best_book_id               10000 non-null  int64  
 3   work_id                    10000 non-null  int64  
 4   books_count                10000 non-null  int64  
 5   isbn                       9300 non-null   object 
 6   isbn13                     9415 non-null   float64
 7   authors                    10000 non-null  object 
 8   original_publication_year  9979 non-null   float64
 9   original_title             9415 non-null   object 
 10  title                      10000 non-null  object 
 11  language_code              8916 non-null   object 
 12  average_rating             10000 non-null  float64
 13  ratings_count              10000 non-null  int6

In [108]:
goodbooks.describe()

,id,book_id,best_book_id,work_id,books_count,isbn13,original_publication_year,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5
count,10000.00000,1.000000e+04,1.000000e+04,1.000000e+04,10000.000000,9.415000e+03,9979.000000,10000.000000,1.000000e+04,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,1.000000e+04,1.000000e+04
mean,5000.50000,5.264697e+06,5.471214e+06,8.646183e+06,75.712700,9.755044e+12,1981.987674,4.002191,5.400124e+04,5.968732e+04,2919.955300,1345.040600,3110.885000,11475.893800,1.996570e+04,2.378981e+04
std,2886.89568,7.575462e+06,7.827330e+06,1.175106e+07,170.470728,4.428619e+11,152.576665,0.254427,1.573700e+05,1.678038e+05,6124.378132,6635.626263,9717.123578,28546.449183,5.144736e+04,7.976889e+04
min,1.00000,1.000000e+00,1.000000e+00,8.700000e+01,1.000000,1.951703e+08,-1750.000000,2.470000,2.716000e+03,5.510000e+03,3.000000,11.000000,30.000000,323.000000,7.500000e+02,7.540000e+02
25%,2500.75000,4.627575e+04,4.791175e+04,1.008841e+06,23.000000,9.780316e+12,1990.000000,3.850000,1.356875e+04,1.543875e+04,694.000000,196.000000,656.000000,3112.000000,5.405750e+03,5.334000e+03
50%,5000.50000,3.949655e+05,4.251235e+05,2.719524e+06,40.000000,9.780452e+12,2004.000000,4.020000,2.115550e+04,2.383250e+04,1402.000000,391.000000,1163.000000,4894.000000,8.269500e+03,8.836000e+03
75%,7500.25000,9.382225e+06,9.636112e+06,1.451775e+07,67.000000,9.780831e+12,2011.000000,4.180000,4.105350e+04,4.591500e+04,2744.250000,885.000000,2353.250000,9287.000000,1.602350e+04,1.730450e+04
max,10000.00000,3.328864e+07,3.553423e+07,5.639960e+07,3455.000000,9.790008e+12,2017.000000,4.820000,4.780653e+06,4.942365e+06,155254.000000,456191.000000,436802.000000,793319.000000,1.481305e+06,3.011543e+06


In [107]:
goodbooks_cols = list(goodbooks.columns)
print("Columns: ", goodbooks_cols)

Columns:  ['id', 'book_id', 'best_book_id', 'work_id', 'books_count', 'isbn', 'isbn13', 'authors', 'original_publication_year', 'original_title', 'title', 'language_code', 'average_rating', 'ratings_count', 'work_ratings_count', 'work_text_reviews_count', 'ratings_1', 'ratings_2', 'ratings_3', 'ratings_4', 'ratings_5']


In [106]:
goodbooks.head()

,id,book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,eng,4.34,4780653,4942365,155254,66715,127936,560092,1481305,2706317
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPrÃ©",1997.0,Harry Potter and the Philosopher's Stone,...,eng,4.44,4602479,4800065,75867,75504,101676,455024,1156318,3011543
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,en-US,3.57,3866839,3916824,95009,456191,436802,793319,875073,1355439
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,eng,4.25,3198671,3340896,72586,60427,117415,446835,1001952,1714267
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,eng,3.89,2683664,2773745,51992,86236,197621,606158,936012,947718


In [104]:
#Dropping the URL columns for the Books table
goodbooks.drop(['image_url', 'small_image_url'], axis=1, inplace=True)
goodbooks.head()

,id,book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,eng,4.34,4780653,4942365,155254,66715,127936,560092,1481305,2706317
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPrÃ©",1997.0,Harry Potter and the Philosopher's Stone,...,eng,4.44,4602479,4800065,75867,75504,101676,455024,1156318,3011543
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,en-US,3.57,3866839,3916824,95009,456191,436802,793319,875073,1355439
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,eng,4.25,3198671,3340896,72586,60427,117415,446835,1001952,1714267
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,eng,3.89,2683664,2773745,51992,86236,197621,606158,936012,947718


In [105]:
print(goodbooks.columns)

Index(['id', 'book_id', 'best_book_id', 'work_id', 'books_count', 'isbn',
       'isbn13', 'authors', 'original_publication_year', 'original_title',
       'title', 'language_code', 'average_rating', 'ratings_count',
       'work_ratings_count', 'work_text_reviews_count', 'ratings_1',
       'ratings_2', 'ratings_3', 'ratings_4', 'ratings_5'],
      dtype='object')


In [111]:
goodbooks.head()

,id,book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,eng,4.34,4780653,4942365,155254,66715,127936,560092,1481305,2706317
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPrÃ©",1997.0,Harry Potter and the Philosopher's Stone,...,eng,4.44,4602479,4800065,75867,75504,101676,455024,1156318,3011543
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,en-US,3.57,3866839,3916824,95009,456191,436802,793319,875073,1355439
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,eng,4.25,3198671,3340896,72586,60427,117415,446835,1001952,1714267
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,eng,3.89,2683664,2773745,51992,86236,197621,606158,936012,947718


In [112]:
#Renaming the columns
goodbooks.rename(columns={'book_id': 'BookID', 'isbn': 'ISBN', 'authors': 'Author', 'original_publication_year': 'Year', 'original_title': 'Title', 'average_rating': 'Avgrating'}, inplace=True)
#Printing the Books Table with renamed columns
print(goodbooks.head())

   id   BookID  best_book_id  work_id  books_count       ISBN        isbn13  \
0   1  2767052       2767052  2792775          272  439023483  9.780439e+12   
1   2        3             3  4640799          491  439554934  9.780440e+12   
2   3    41865         41865  3212258          226  316015849  9.780316e+12   
3   4     2657          2657  3275794          487   61120081  9.780061e+12   
4   5     4671          4671   245494         1356  743273567  9.780743e+12   

                         Author    Year  \
0               Suzanne Collins  2008.0   
1  J.K. Rowling, Mary GrandPrÃ©  1997.0   
2               Stephenie Meyer  2005.0   
3                    Harper Lee  1960.0   
4           F. Scott Fitzgerald  1925.0   

                                      Title  ... language_code Avgrating  \
0                          The Hunger Games  ...           eng      4.34   
1  Harry Potter and the Philosopher's Stone  ...           eng      4.44   
2                                  Twi

In [116]:
goodbooks.loc[goodbooks['Year'].isnull()]

,id,BookID,best_book_id,work_id,books_count,ISBN,isbn13,Author,Year,Title,...,language_code,Avgrating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5
219,220,3609760,3609760,3652511,20,316043133,9.780316e+12,Mark Cotta Vaz,NaN,Twilight: The Complete Illustrated Movie Compa...,...,en-US,4.23,291411,291899,532,6862,11019,48008,67939,158071
975,976,90072,90072,24501333,3,793551617,9.780794e+12,"Robert Kapilow, Dr. Seuss",NaN,NaN,...,NaN,4.44,90574,90691,262,803,1943,10781,20483,56681
3505,3506,856917,856917,3874446,9,1421500167,9.781422e+12,Yuu Watase,NaN,çµ¶å¯¾å½¼æ° (Zettai Kareshi) 1,...,eng,4.01,29385,29550,400,1197,2156,5661,6559,13977
4228,4229,36,36,4414,12,618391002,9.780618e+12,"Chris Smith, Christopher Lee, Richard Taylor",NaN,The Lord of the Rings: Weapons and Warfare,...,eng,4.53,18788,18911,49,297,282,1556,3743,13033
4247,4248,8724779,8724779,13597737,1,312530641,9.780313e+12,"P.C. Cast, Kristin Cast",NaN,NaN,...,eng,4.43,20578,20578,77,471,648,2118,3645,13696
4409,4410,349254,349254,17150,10,618390979,9.780618e+12,Jude Fisher,NaN,NaN,...,eng,4.59,17526,17641,28,157,202,1259,3458,12565
4707,4708,9832370,9832370,27881851,1,NaN,NaN,BookRags,NaN,NaN,...,eng,4.59,18960,18960,92,278,151,877,4460,13194
4770,4771,5171894,5171894,5238959,50,61231401,9.780061e+12,James Rollins,NaN,NaN,...,en-US,4.16,19510,21831,864,169,555,3623,8819,8665
4877,4878,17733861,17733861,24809815,11,NaN,NaN,Bella Forrest,NaN,NaN,...,eng,4.01,24757,25111,1302,437,1315,5250,8685,9424
5609,5610,1044141,1044141,1673783,68,743287932,9.780743e+12,Stephen R. Covey,NaN,NaN,...,en-US,3.99,13765,14396,330,319,789,2989,4904,5395


In [135]:
average_year = goodbooks.loc[goodbooks['Year'] >= 1800, 'Year'].mean().round()

In [117]:
goodbooks['Year'].unique()

array([ 2008.,  1997.,  2005.,  1960.,  1925.,  2012.,  1937.,  1951.,
        2000.,  1813.,  2003.,  2011.,  1949.,  1945.,  1947.,  2009.,
        1999.,  1954.,  2010.,  2002.,  1998.,  2007.,  1595.,  1988.,
        1993.,  1950.,  1996.,  2006.,  1868.,  1847.,  2001.,  1953.,
        1974.,  1979.,  1932.,  1884.,  1952.,  2015.,  1995.,  2004.,
        1969.,  1936.,  1985.,  1818.,  1977.,  1811.,  -720.,  1946.,
        1859.,  1990.,  1964.,  1989.,  1958.,  1973.,  1967.,  1911.,
        1891.,  1897.,  1963.,  1844.,  2013.,  1862.,  1961.,  1876.,
        1962.,  1955.,  1991.,  1600.,  1965.,  1939.,  1908.,  1850.,
        2014.,  1606.,  1860.,  1942.,  1978.,  1815.,  1877.,  1986.,
        1866.,  1922.,  1987.,  1851.,  1982.,  1843.,  1976.,  1994.,
        1915.,  1956.,    nan,  1980.,  1865.,  1817.,  1957.,  1926.,
        1943.,  1938.,  1966.,  1981.,  2016.,  1992.,  1984.,  1972.,
        1882.,  1895.,  1899.,  1983.,  -750.,  1900.,  1975.,  1971.,
      

In [121]:
# goodcount = goodbooks['Year'].value_counts()
# goodmaxyear = goodcount[goodcount == goodcount.max()].index.tolist()
# print(goodmaxyear)

[2012.0]


In [145]:
goodbooks.loc[books['Year'] == 0, 'Year'] = average_year
goodbooks.loc[:, 'Year'] = goodbooks['Year'].fillna(average_year)
goodbooks.loc[goodbooks['Year'] < 1800, 'Year'] = average_year


In [146]:
print(goodbooks.loc[goodbooks['Year']==0])
print(goodbooks.loc[goodbooks['Year'].isnull()])


Empty DataFrame
Columns: [id, BookID, best_book_id, work_id, books_count, ISBN, isbn13, Author, Year, Title, title, language_code, Avgrating, ratings_count, work_ratings_count, work_text_reviews_count, ratings_1, ratings_2, ratings_3, ratings_4, ratings_5]
Index: []

[0 rows x 21 columns]
Empty DataFrame
Columns: [id, BookID, best_book_id, work_id, books_count, ISBN, isbn13, Author, Year, Title, title, language_code, Avgrating, ratings_count, work_ratings_count, work_text_reviews_count, ratings_1, ratings_2, ratings_3, ratings_4, ratings_5]
Index: []

[0 rows x 21 columns]


In [147]:
print(goodbooks.loc[goodbooks['Year']<1800])

Empty DataFrame
Columns: [id, BookID, best_book_id, work_id, books_count, ISBN, isbn13, Author, Year, Title, title, language_code, Avgrating, ratings_count, work_ratings_count, work_text_reviews_count, ratings_1, ratings_2, ratings_3, ratings_4, ratings_5]
Index: []

[0 rows x 21 columns]


In [151]:
goodbooks.dropna(subset=['Title'], inplace=True)

In [152]:
goodbooks['Title'].isnull().sum()

0

In [159]:
goodbooks.duplicated().sum()

0

In [156]:
#creating a new data frame only with the relevant columns
selected_cols = ['id', 'BookID', 'Author', 'Year', 'Title', 'Avgrating', 'ratings_count']
new_goodbooks = goodbooks[selected_cols].copy()

In [160]:
new_goodbooks.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9415 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             9415 non-null   int64  
 1   BookID         9415 non-null   int64  
 2   Author         9415 non-null   object 
 3   Year           9415 non-null   float64
 4   Title          9415 non-null   object 
 5   Avgrating      9415 non-null   float64
 6   ratings_count  9415 non-null   int64  
dtypes: float64(2), int64(3), object(2)
memory usage: 588.4+ KB


In [157]:
new_goodbooks.head()

,id,BookID,Author,Year,Title,Avgrating,ratings_count
0,1,2767052,Suzanne Collins,2008.0,The Hunger Games,4.34,4780653
1,2,3,"J.K. Rowling, Mary GrandPrÃ©",1997.0,Harry Potter and the Philosopher's Stone,4.44,4602479
2,3,41865,Stephenie Meyer,2005.0,Twilight,3.57,3866839
3,4,2657,Harper Lee,1960.0,To Kill a Mockingbird,4.25,3198671
4,5,4671,F. Scott Fitzgerald,1925.0,The Great Gatsby,3.89,2683664


In [162]:
new_goodbooks.describe()

,id,BookID,Year,Avgrating,ratings_count
count,9415.000000,9.415000e+03,9415.000000,9415.000000,9.415000e+03
mean,4917.828784,4.876324e+06,1993.444291,3.998797,5.568348e+04
std,2881.703907,7.328432e+06,29.589444,0.252048,1.615800e+05
min,1.000000,1.000000e+00,1807.000000,2.470000,2.716000e+03
25%,2414.500000,4.378150e+04,1990.500000,3.850000,1.372800e+04
50%,4882.000000,3.326130e+05,2004.000000,4.010000,2.164000e+04
75%,7399.500000,8.099727e+06,2010.000000,4.170000,4.242350e+04
max,10000.000000,3.328864e+07,2017.000000,4.820000,4.780653e+06


In [164]:
new_goodbooks.shape

(9415, 7)

In [158]:
new_goodbooks.isnull().sum()

id               0
BookID           0
Author           0
Year             0
Title            0
Avgrating        0
ratings_count    0
dtype: int64